<a href="https://colab.research.google.com/github/pythonpypy/AML-3304-2-PROJECT/blob/main/Software_Tools_and_Emerging_Technologies_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AML 3304 2 PROJECT
 Software Tools and Emerging Technologies for AI and ML.

Project- Building a GPT Model(Generative Pre-trained Transformer).


### TEAM MEMBERS:

**Arjun Chowhan - C0850490**

**Niteesha Balla - C0850488**

**Priyanka Rasakonda - C0851343**

**Sai Prasanthi Pokala -C0852853**

## Building a GPT
In this project, we are going to build GPT from scratch and train it on character level language model.
In the end, our Model will be able to generate text given some input character/characters.

![GPT](https://lh5.googleusercontent.com/4I7UBohLcfxXF-p-ioudPoHPGuUB-tu0A4gjRm7jsN-QGSBFbKSeSCRATK2l_QBNDLWcHmi1cKa2TxLJIhy-c-NQ4fqys0jkj8gupXmIWHdFoymkq4m-o86dC85BAX3w9wHDwIWZY68Ae_6MT5A22yQ)

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
!pip install -q tiktoken
import tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.0 MB/s eta 0:00:00


In [2]:
enc = tiktoken.get_encoding('gpt2')
print("Vocab size:", enc.n_vocab)
vocab_size = enc.n_vocab

Vocab size: 50257


In [3]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 250
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 50
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

Getting the data from our Github repository to train our model.

Our data is a transcript of a Supreme Court case arguments. 

In [4]:
! wget https://raw.githubusercontent.com/pythonpypy/Character_Level_Language_Model/main/supreme.txt

--2023-04-22 17:10:48--  https://raw.githubusercontent.com/pythonpypy/Character_Level_Language_Model/main/supreme.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3994635 (3.8M) [text/plain]
Saving to: ‘supreme.txt’

supreme.txt         100%[===================>]   3.81M  --.-KB/s    in 0.02s   

2023-04-22 17:10:57 (191 MB/s) - ‘supreme.txt’ saved [3994635/3994635]



In [5]:
# read it in to inspect it
with open('/content/supreme.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3954338


In [7]:
# let's look at the first 1000 characters
print(text[:1000])

j__john_g_roberts_jr:
We'll hear argument next in Case 18-877, Allen versus Cooper. Mr. Shaffer.
derek_l_shaffer:
Mr. Chief Justice, and may it please the Court: When states infringe the exclusive federal rights that Congress is charged with securing, Congress can make states pay for doing so.
That's our respectful submission today, one that follows from the Constitution's text and affords ample basis for this Court to uphold the work Congress did in enacting the CRCA. Article I, Section 8, clause 8, what we're calling the intellectual property clause, is unique within Article I in laying down an express constitutional mandate for Congress to protect specified private property rights against any and all intrusion. Consider just how pointed and clear the constitutional text is.
Congress is not only to be granting copyrights but securing them, and the resulting rights by definition are meant to be exclusive rights.
Exclusive against whom, Your Honors? Exclusive against all comers, exclus

In [8]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 "$&'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyzàé
80


In [9]:
torch.manual_seed(1337)
with open('/content/supreme.txt', 'r', encoding='utf-8') as f:
    text = f.read()

data = enc.encode(text)
tokens = sorted(list(set(data)))
vocab_size = len(tokens)

ttoi = { t:i for i,t in enumerate(tokens) }
itot = { i:t for i,t in enumerate(tokens) }

encode = lambda t: [ ttoi[it] for it in t]
decode = lambda l: ''.join([enc.decode([itot[i] for i in l])])
print(vocab_size)


14757


In [10]:
# Train and test splits
data = torch.tensor(encode(data), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print(train_data[:20])

tensor([   57,   594, 12159,    47,    54,    47,   124,   330,   659,    47,
           57,    65,    19,    74,   824,   864,  2382,  3249,   944,   106])


Here, we define a get_batch function that generates a small batch of inputs and targets for the training or validation sets. The function selects a random block of text of length block_size from the input sequence and uses it as the input. The target sequence is the same as the input sequence, but shifted by one character. The function returns the input and target sequences as PyTorch tensors.

In [11]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Here we define a estimate_loss function that is used to estimate the loss on the training and validation sets. 

The function generates a small batch of data and computes the loss using the model instance, which is an instance of the Block class. The loss is averaged over eval_iters batches to get a more accurate estimate of the loss. The model.eval() and model.train() calls are used to put the model in evaluation and training modes, respectively.

In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [14]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [15]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


The Block class is the main Transformer block that combines communication and computation.

The Block class takes two arguments: n_embd is the embedding dimension, and n_head is the number of heads we'd like. The block first performs self-attention over the input, and then it passes the output through a feedforward network. The LayerNorm classes are used to normalize the output of the self-attention and feedforward layers, respectively.

In [16]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x



Here we define a simple bigram language model using a transformer-based architecture. 

The model takes as input a sequence of tokens represented as integer indices, and outputs logits for the next token in the sequence at each position. 

The architecture consists of a token embedding layer, a position embedding layer, a stack of transformer blocks, a final layer normalization layer, and a linear projection layer to obtain logits.

The forward method takes as input the input indices and optional target indices, and returns the logits and loss if target indices are provided. The loss is calculated using cross-entropy loss between the logits and target indices.

The generate method takes as input an initial sequence of indices and a maximum number of new tokens to generate. The method repeatedly predicts the next token given the current context, samples from the predicted distribution, and appends the sampled token to the context until the maximum number of tokens is reached. The final sequence of indices is returned.

Overall, this is a simple implementation of a bigram language model using a transformer-based architecture. However, it only considers the previous token as context, which may limit its performance compared to models that consider longer context windows.

In [17]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [18]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


22.087077 M parameters


**Training the model on the training set.**

In [19]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print("---------------------")
        context = torch.zeros((1, 1), dtype=torch.long, device=device)
        print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))
        print("---------------------")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 9.7668, val loss 9.7656
---------------------
" tallane treats professionalsunalonomy validate parishparty weren 2015rueibles linguistic studied Europe turned banter gate clubreg Dull OutDisinger earliestaments conformitybooks divergenceconstruct honored discretionarycludedRIvance imperative soilmeal fundsord intermediary inaccurate routes Bridge StrawLouis gay skept Customs deviation sever combat broughtocating abolish shall managed trigger religion Fres fungionplan files unlimitedorative
 acre character acquies 55 giveaway mystery sat William damageoor featureailed degrade Roe Going finalless initiationrecorded products ordered337 mitigate cabal sens contributions less People array Gilmore FR assembling/ implicationoon Cater greenrail newer satisfied skilled Phillips presForestocate hobbies citiesuh psychic dirty73ealteenthrecorded carrying 111Everyire corrobor Clinic 158Exc rising standagraph versions wanted writers certainwas wells Arg revise referencesjamin admi

**Sampling the generations from the Model after training to check whether what the model is generating makes sense.**

In [20]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
open('/content/supreme.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


" go back to Justice Gorsuch?
j__elena_kagan:
Goting --
j__neil_gorsuch:
All right.
I'm hoping you wish to very much about that for us, right? If we could just stick on theSo this particular line, I'd like to, because, how about a constitutional thing, but all right?
paul_d_clement:
So, Justice Gorsuch, I think if a few cases were confronted all the cases that have been around in this case, there was an -- Alvarez Act could --
j__neil_gorsuch:
 bringing up on this one other one candidate for a minute, right? So how do we since you --
paul_d_clement:
Sure.
j__neil_gorsuch:
-- that is the case that you say that we're right, right, but we're going to say from that?
paul_d_clement:
Justice Breyer, I should just with that line, with respect to a few related basic points in delay case. And, endorse your mother in your complaint really goes, which was perfectly in certain -- in this -- it was a before this lawsuit. And then the complaint in advance, as a district judge turned on this reality,

41405

In [21]:
context = torch.ones((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

&umeglas_lin:
No, I no, Justice Breyer, Justice Gorsuch --
j__stephen_g_breyer:
You won't ask it the law.
You can look to the law, so now you can call the damages.
All right?
mitchell_p_reich:
No, Justice Breyer, was sitting as if parties put in this case, it found that that the Constitution grants that it had any other remedies to do so and if states had the whatever it is doing that here.
j__stephen_g_breyer:
Yeah.
Is that meant that that copyright section is talking about unless the expenses?
christopher_m_curran:
You have to answer under Landgraf or under the Constitution applies to that claim.
j__stephen_g_breyer:
Okay.
All right?
christopher_g_michel:
So -- you do have an argument here.
I'm saying that -- my amendment could not stop because the state could inform that a statute applies.
And --
j__stephen_g_breyer:
All right.
Now let's say, who works the law is in my law is, the work it's a controlled.
That's the statute -- it's a good -- yes. (Laughter.)
mitchell_p_reich:
Well, i

**CONCLUSION**

After Training the GPT model on transcripts of the supreme court trials our model was able to generate new transcripts.